In [6]:
import sys
sys.path.append('F:\Cambridge\Project\MHMC-for-VAE/hmc')
from hmc_base import *
from hmc_unconstrained import *
import autograd.numpy as np
import matplotlib.pyplot as plt
from scipy.special import beta, comb, gamma
from scipy.stats import multivariate_normal
import operator as op
import torch
#from autograd.numpy import inf

In [3]:
# get data pair list
file = open("cancermortality.txt", "r") 
data = (file.read()) 
tmp = data.split()
tmp = tmp[2:len(tmp)]
tmp = [int(e) for e in tmp]
data_list = []
for i in range(0, int(len(tmp)-1), 2):
    data_list.append([tmp[i], tmp[i+1]])

In [4]:
# get death number list
death = []
for i in range(len(data_list)):
    death.append(data_list[i][0])
# convert to np array
death = np.array(death)

# get risk number list
risk = []
for i in range(len(data_list)):
    risk.append(data_list[i][1])
# convert to np array
risk = np.array(risk)

In [5]:
def lower_bound(y, n, mass_matrix, n_steps, z0_mean, z0_cov, v1_p_mean, v1_p_cov, v1_mean, v1_cov, joint_function):
    # sample epsilson
    eps_mean1 = np.array([0, 0])
    eps_cov1 = np.identity(2)
    eps1 = np.random.multivariate_normal(eps_mean1, eps_cov1)
    
    eps_mean2 = np.array([0, 0])
    eps_cov2 = np.identity(2)
    eps2 = np.random.multivariate_normal(eps_mean2, eps_cov2)
    
    eps_mean3 = np.array([0, 0])
    eps_cov3 = np.identity(2)
    eps3 = np.random.multivariate_normal(eps_mean3, eps_cov3)
    
    # Draw an initial random variable z0 
    z0 = z0_mean + np.sqrt(z0_cov).dot(eps1)
    #eta0 = (np.tanh(z0[0])+1)/200 #range(0, 0.01)
    eta0 = z0[0]
    k0 = z0[1]
    
    while((eta0<0) or (eta0>0.01) or (k0<=0)):
        eps1 = np.random.multivariate_normal(eps_mean1, eps_cov1)
        z0 = z0_mean + np.sqrt(z0_cov).dot(eps1)
        eta0 = z0[0]
        k0 = z0[1]
    
    print(eta0, k0)
    # Init. lower bound
    theta0 = torch.tensor([eta0, k0], requires_grad=True)
    joint_t0 = joint_function(theta, y, n)
    
    L1 = torch.log(joint_t0)
    L2 = torch.log(multivariate_normal.pdf(z0, z0_mean, z0_cov))
    L = L1 - L2
    
    # define energy function
    def posterior_energy(theta, cache):
        eta = theta[0]
        k = theta[1]
        post = posterior(y, n, eta, k)
        return -np.log(post)
    
    # MCMC steps
    for i in range(1):
        
        grad_test = grad(joint_function)       # Obtain its gradient function
        tt = grad_test(np.array([eta0, k0]),y,n) 
        print(str(tt))
        # Draw vt'
        vt_p = v1_p_mean + np.sqrt(v1_p_cov).dot(eps2)
        
        #init hmc sampler
        sampler = IsotropicHmcSampler(posterior_energy, energy_grad=None, prng=None,
                 mom_resample_coeff=0., dtype=np.float64)
        pos_samples, mom_samples, ratio = sampler.get_samples(np.array([eta0, k0]), 0.1, n_steps, 1, mass_matrix)
        z1 = pos_samples[0]
        v1 = mom_samples[0]
        #eta1 = z1[0]
        #k1 = z1[1]
        eta1 = z1[0]
        k1 = z1[1]
        
        while((eta1<0) or (eta1>0.01) or (k1<=0)):
            pos_samples, mom_samples, ratio = sampler.get_samples(np.array([eta0, k0]), 0.1, n_steps, 1, mass_matrix)
            z1 = pos_samples[0]
            v1 = mom_samples[0]
            eta1 = z1[0]
            k1 = z1[1]
        
        print(eta1, k1)
        joint_t1 = joint_function(np.array([eta1, k1]), y, n)
        rt = multivariate_normal.pdf(v1, v1_mean, v1_cov)
        qt = multivariate_normal.pdf(vt_p, v1_p_mean, v1_p_cov)
        
        alpha = (joint_t1*rt)/(joint_t0*qt)
        L = L + np.log(alpha)
    
    
    
    return pos_samples, L

In [54]:
def lbeta(a, b):
    return torch.lgamma(a) + torch.lgamma(b) - torch.lgamma(a+b)

def log_beta(a,b):
    
def prior_beta_binomial(theta):
    eta = theta[0]
    k = theta[1]
    t1 = 1/(eta*(1-eta))
    t2 = 1/(1+k)**2
    return t1*t2

def likelihood_beta_binomial(theta, y, n):
    eta = theta[0]
    k = theta[1]
    n_data = y.shape[0]
    result = 0.
    for i in range(n_data):
        n_comb = torch.tensor([comb(n[i], y[i])], requires_grad=True)
        #one_likelihood = beta(k*eta+y[i], k*(1-eta)+n[i]-y[i])/beta(k*eta, k*(1-eta))
        y_tensor = torch.tensor([float(y[i])], requires_grad=True)
        n_tensor = torch.tensor([float(n[i])], requires_grad=True)
        #one_likelihood = lbeta(torch.tensor([k*eta+y_tensor]), torch.tensor([k*(1-eta)+n_tensor-y_tensor]))-lbeta(torch.tensor([k*eta]), torch.tensor([k*(1-eta)]))
        one_likelihood = lbeta((k*eta+y_tensor), (k*(1-eta)+n_tensor-y_tensor))-lbeta((k*eta), (k*(1-eta)))
        one_likelihood = torch.log(n_comb)+one_likelihood
        lbeta((k*eta+y_tensor), (k*(1-eta)+n_tensor-y_tensor))
        print((k*eta+y_tensor))
        print((k*(1-eta)+n_tensor-y_tensor))
        print(lbeta((k*eta), (k*(1-eta))))
        #print(beta(k*eta+y[i], k*(1-eta)+n[i]-y[i]))
        #print(beta(k*eta, k*(1-eta)))
        print("===================")
        print(one_likelihood)
        result = result+one_likelihood
    return result

def joint_beta_binomial(theta, y, n):
    #eta = theta[0]
    #k = theta[1]
    prior = prior_beta_binomial(theta)
    print("prior "+str(prior))
    likelihood = likelihood_beta_binomial(theta, y, n)
    print("like "+str(likelihood))
    return prior + likelihood

def posterior(y, n, eta, k):
    t1 = 1/(eta*(1-eta))
    t2 = 1/np.power((1+k), 2)
    t3 = np.prod(beta(k*eta+y, k*(1-eta)+n-y)/beta(k*eta, k*(1-eta)))
    final = t1*t2*t3
    return final

In [43]:
def lower_bound(y, n, mass_matrix, n_steps, z0_mean, z0_cov, a1, b1, a2, b2, joint_function):
    # sample epsilson
    eps_mean1 = np.array([0, 0])
    eps_cov1 = np.identity(2)
    eps1 = np.random.multivariate_normal(eps_mean1, eps_cov1)
    
    eps_mean2 = np.array([0, 0])
    eps_cov2 = np.identity(2)
    eps2 = np.random.multivariate_normal(eps_mean2, eps_cov2)
    
    eps_mean3 = np.array([0, 0])
    eps_cov3 = np.identity(2)
    eps3 = np.random.multivariate_normal(eps_mean3, eps_cov3)
    
    # Draw an initial random variable z0 
    z0 = z0_mean + torch.mm(torch.sqrt(z0_cov), eps1)
    #eta0 = (np.tanh(z0[0])+1)/200 #range(0, 0.01)
    eta0 = z0[0]
    k0 = z0[1]
    
    while((eta0.item()<0) or (eta0.item()>0.01) or (k0.item()<=0)):
        eps1 = np.random.multivariate_normal(eps_mean1, eps_cov1)
        z0 = z0_mean + torch.mm(torch.sqrt(z0_cov), eps1)
        eta0 = z0[0]
        k0 = z0[1]
    
    # Init. lower bound
    theta0 = torch.tensor([eta0, k0], requires_grad=True)
    joint_t0 = joint_function(theta, y, n)
    
    L1 = torch.log(joint_t0)
    L2 = torch.log(torch.tensor(multivariate_normal.pdf(z0.item(), z0_mean.item(), z0_cov.item())))
    L = L1 - L2
    
    # define energy function
    def posterior_energy(theta, cache):
        eta = theta[0]
        k = theta[1]
        post = posterior(y, n, eta, k)
        return -np.log(post)
    
    # MCMC steps
    for i in range(1):
        
        grad_test = grad(joint_function)       # Obtain its gradient function
        tt = grad_test(np.array([eta0, k0]),y,n) 
        print(str(tt))
        # Draw vt'
        vt_p = v1_p_mean + np.sqrt(v1_p_cov).dot(eps2)
        
        #init hmc sampler
        sampler = IsotropicHmcSampler(posterior_energy, energy_grad=None, prng=None,
                 mom_resample_coeff=0., dtype=np.float64)
        pos_samples, mom_samples, ratio = sampler.get_samples(np.array([eta0, k0]), 0.1, n_steps, 1, mass_matrix)
        z1 = pos_samples[0]
        v1 = mom_samples[0]
        #eta1 = z1[0]
        #k1 = z1[1]
        eta1 = z1[0]
        k1 = z1[1]
        
        while((eta1<0) or (eta1>0.01) or (k1<=0)):
            pos_samples, mom_samples, ratio = sampler.get_samples(np.array([eta0, k0]), 0.1, n_steps, 1, mass_matrix)
            z1 = pos_samples[0]
            v1 = mom_samples[0]
            eta1 = z1[0]
            k1 = z1[1]
        
        print(eta1, k1)
        joint_t1 = joint_function(np.array([eta1, k1]), y, n)
        rt = multivariate_normal.pdf(v1, v1_mean, v1_cov)
        qt = multivariate_normal.pdf(vt_p, v1_p_mean, v1_p_cov)
        
        alpha = (joint_t1*rt)/(joint_t0*qt)
        L = L + np.log(alpha)
    
    
    
    return pos_samples, L

In [16]:
x =  torch.tensor([5.], requires_grad=True)
y = np.array([3.])
torch.log(torch.from_numpy(y))

tensor([ 1.0986], dtype=torch.float64)

In [32]:
x =  torch.tensor([[0., 1.],[1., 0.]], requires_grad=True)
multivariate_normal.pdf(z0.item(), z0_mean.item(), z0_cov.item())

NameError: name 'z0' is not defined

In [31]:
x =  torch.tensor([0.], requires_grad=True)
y = torch.tensor([0.], requires_grad=True)
z = torch.tensor([1.], requires_grad=True)

tensor(1.00000e-02 *
       [ 6.4176])

In [55]:
def lower_bound(y, n, mass_matrix, n_steps, z0_mean, z0_cov, a1, b1, v1_p_cov, a2, b2, v1_cov, joint_function):
    # sample epsilson
    eps_mean1 = np.array([0, 0])
    eps_cov1 = np.identity(2)
    eps1 = np.random.multivariate_normal(eps_mean1, eps_cov1)
    
    eps_mean2 = np.array([0, 0])
    eps_cov2 = np.identity(2)
    eps2 = np.random.multivariate_normal(eps_mean2, eps_cov2)
    
    eps_mean3 = np.array([0, 0])
    eps_cov3 = np.identity(2)
    eps3 = np.random.multivariate_normal(eps_mean3, eps_cov3)
    
    # Draw an initial random variable z0 
    z0 = z0_mean + np.sqrt(z0_cov).dot(eps1)
    #eta0 = (np.tanh(z0[0])+1)/200 #range(0, 0.01)
    eta0 = z0[0]
    k0 = z0[1]
    
    while((eta0<0) or (eta0>0.01) or (k0<=0)):
        eps1 = np.random.multivariate_normal(eps_mean1, eps_cov1)
        z0 = z0_mean + np.sqrt(z0_cov).dot(eps1)
        eta0 = z0[0]
        k0 = z0[1]
    
    print(eta0, k0)
    # Init. lower bound
    theta0 = torch.tensor([eta0, k0], requires_grad=True)
    joint_t0 = joint_function(theta0, y, n)
    
    # gradient
    joint_t0.backward()
    theta0_grad = theta0.grad.numpy()
    
    L1 = np.log(joint_t0.item())
    L2 = np.log(multivariate_normal.pdf(z0, z0_mean, z0_cov))
    L = L1 - L2
    
    # define energy function
    def posterior_energy(theta, cache):
        eta = theta[0]
        k = theta[1]
        post = posterior(y, n, eta, k)
        return -np.log(post)
    
    # MCMC steps
    for i in range(1):
        
        #grad_test = grad(joint_function)       # Obtain its gradient function
        #tt = grad_test(np.array([eta0, k0]),y,n) 
         
        v1_p_mean = a1*z0 + b1*theta0_grad # mean of guassian
        # Draw vt'
        vt_p = v1_p_mean + np.sqrt(v1_p_cov).dot(eps2)
        
        #init hmc sampler
        sampler = IsotropicHmcSampler(posterior_energy, energy_grad=None, prng=None,
                 mom_resample_coeff=0., dtype=np.float64)
        pos_samples, mom_samples, ratio = sampler.get_samples(np.array([eta0, k0]), 0.1, n_steps, 1, mass_matrix)
        z1 = pos_samples[0]
        v1 = mom_samples[0]
        #eta1 = z1[0]
        #k1 = z1[1]
        eta1 = z1[0]
        k1 = z1[1]
        
        while((eta1<0) or (eta1>0.01) or (k1<=0)):
            pos_samples, mom_samples, ratio = sampler.get_samples(np.array([eta0, k0]), 0.1, n_steps, 1, mass_matrix)
            z1 = pos_samples[0]
            v1 = mom_samples[0]
            eta1 = z1[0]
            k1 = z1[1]
        
        print(eta1, k1)
        
        theta1 = torch.tensor([eta1, k1], requires_grad=True)
        joint_t1 = joint_function(theta1, y, n)
        # gradient
        joint_t1.backward()
        theta1_grad = theta1.grad.numpy()
        
        #joint_t1 = joint_function(np.array([eta1, k1]), y, n)
        v1_mean = a2*z1 + b2*theta1_grad # mean of guassian
        rt = multivariate_normal.pdf(v1, v1_mean, v1_cov)
        qt = multivariate_normal.pdf(vt_p, v1_p_mean, v1_p_cov)
        
        alpha = (joint_t1.item()*rt)/(joint_t0.item()*qt)
        L = L + np.log(alpha)
        
    return pos_samples, L

In [49]:
mass = 1.
mass_matrix = mass*np.identity(2)
z0_mean = np.array([0.0001,1000]) 
z0_cov = np.array([[1,0],[0,10]])
v1_p_mean = np.array([0,0])
v1_p_cov = np.identity(2)
v1_mean = np.array([0,0])
v1_cov = np.identity(2)

In [ ]:
pos_samples, L = lower_bound(death, risk, mass_matrix, 5, z0_mean, z0_cov, v1_p_mean, v1_p_cov, v1_mean, v1_cov, joint_beta_binomial)

In [56]:
pos_samples, L = lower_bound(death, risk, mass_matrix, 5, z0_mean, z0_cov, 1, 1, v1_p_cov, 1, 1, v1_cov, joint_beta_binomial)

0.00753876977123 1002.61587427
prior tensor(1.00000e-04 *
       1.3269)
tensor([ 7.5585])
tensor([ 2078.0574])
tensor(-44.5513)
tensor([-5.5532])
tensor([ 7.5585])
tensor([ 1850.0574])
tensor(-44.5513)
tensor([-4.6763])
tensor([ 9.5585])
tensor([ 4454.0571])
tensor(-44.5513)
tensor([-8.3435])
tensor([ 7.5585])
tensor([ 1652.0574])
tensor(-44.5513)
tensor([-3.8218])
tensor([ 8.5585])
tensor([ 2202.0574])
tensor(-44.5513)
tensor([-4.5727])
tensor([ 8.5585])
tensor([ 2019.0574])
tensor(-44.5513)
tensor([-3.9948])
tensor([ 7.5585])
tensor([ 1522.0574])
tensor(-44.5513)
tensor([-3.2046])
tensor([ 9.5585])
tensor([ 2661.0574])
tensor(-44.5513)
tensor([-4.8838])
tensor([ 8.5585])
tensor([ 1577.0574])
tensor(-44.5513)
tensor([-2.4487])
tensor([ 10.5585])
tensor([ 1574.0574])
tensor(-44.5513)
tensor([-1.8289])
tensor([ 7.5585])
tensor([ 1912.0574])
tensor(-44.5513)
tensor([-4.9243])
tensor([ 8.5585])
tensor([ 1851.0574])
tensor(-44.5513)
tensor([-3.4316])
tensor([ 8.5585])
tensor([ 1674.0574])

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: RuntimeWarning: invalid value encountered in double_scalars


In [48]:
L

nan